# UNIVERSIDAD DE LIMA

## OBTENIENDO LOS ENLACES A LAS TESIS

In [ ]:
uni = "Universidad de Lima"
link = "https://repositorio.ulima.edu.pe/discover?rpp=10&etal=0&scope=20.500.12724/1409&group_by=none&page=1"

In [3]:
import requests
from bs4 import BeautifulSoup

In [ ]:
rpt = requests.get(link)

In [ ]:
soup = BeautifulSoup(rpt.text, "html.parser")

In [ ]:
tesis = soup.find_all(class_="row ds-artifact-item ")

In [ ]:
lista_tesis = []
for div in tesis:
  lista_tesis.append("https://repositorio.ulima.edu.pe/"+f"{div.find('a').get('href')}")

In [ ]:
lista_tesis

['https://repositorio.ulima.edu.pe//handle/20.500.12724/16321',
 'https://repositorio.ulima.edu.pe//handle/20.500.12724/16334',
 'https://repositorio.ulima.edu.pe//handle/20.500.12724/15986',
 'https://repositorio.ulima.edu.pe//handle/20.500.12724/16306',
 'https://repositorio.ulima.edu.pe//handle/20.500.12724/16267',
 'https://repositorio.ulima.edu.pe//handle/20.500.12724/15791',
 'https://repositorio.ulima.edu.pe//handle/20.500.12724/15631',
 'https://repositorio.ulima.edu.pe//handle/20.500.12724/15579',
 'https://repositorio.ulima.edu.pe//handle/20.500.12724/15610',
 'https://repositorio.ulima.edu.pe//handle/20.500.12724/15636']

## OBTENIENDO LA INFORMACIÓN DE UNA TESIS

In [4]:
link_tesis = "https://repositorio.ulima.edu.pe//handle/20.500.12724/16321"
resp_t = requests.get(link_tesis)
soup_t = BeautifulSoup(resp_t.text, "html.parser")

In [20]:
#Obteniendo el título
soup_t.find("h2").text

'Reestructuración del desembarcadero pesquero artesanal de Ilo'

In [6]:
#Nombre del tesista
a = soup_t.find_all(class_ = "simple-item-view-authors")[0].find_all("div")
for autor in a:
  print(autor.text.strip("\n"))

Cruz Falcon, Paul
Maldonado Fuentes, Maria Angela


In [22]:
#Grado
import re
grado = soup_t.find_all(class_="simple-item-view-uri")[1].text
grado_f = re.findall('\[.*\]', grado)[0].strip("[").strip("]")
print(grado_f)

Trabajo de suficiencia profesional para optar el Título Profesional de Arquitecto, Universidad de Lima


In [8]:
#Nombre del asesor(es)
b = soup_t.find_all(class_ = "simple-item-view-authors")[1].find_all("div")
for autor in b:
  print(autor.text.strip("\n"))

Takano Valdivia, Karen


In [9]:
#Año
str(soup_t.find(class_="simple-item-view-date").find_all(text=True, recursive=False)[1])

'2022'

In [10]:
#Resumen
str(soup_t.find(class_ = "simple-item-view-description").find("div").find_all(text=True, recursive=False)[0])

'Actualmente, la infraestructura pesquera artesanal en el país ignora toda relación con el entorno urbano en el que se emplaza. La investigación surge a partir de esta problemática además del cumplimiento de la Norma Sanitaria n°040, la cual estipula lineamientos para una mejor calidad del producto pesquero. Tras la revisión de los DPA’s (Desembarcaderos pesqueros artesanales) actualmente en ejecución/remodelación, se encuentra al DPA de Ilo como el más crítico ya que no cuenta con un área de expansión, posee una infraestructura deficiente y se encuentra en contacto directo con el casco urbano antiguo de la ciudad, desarrollando un importante desafío: la convivencia entre el programa funcional del DPA (flujos productivos pesqueros) y el desarrollo de actividades públicas en un mismo espacio. Con la intención de abrir el puerto hacia la ciudad, sin alterar los flujos productivos pesqueros, surge la idea de continuar recorridos públicos como los que se llevan a cabo en el malecón costero

In [23]:
# Categoría/Subcategoría
cat = soup_t.find_all(class_="simple-item-view-uri")[3].find("span").text
cat

'Arquitectura / Urbanismo'

## OBTENIENDO LA IFORMACIÓN DE TODAS LAS TESIS

In [17]:
n = 1 #Número de páginas a scrapear
lista_tesis = []
for i in range(1,n+1):
  url = f"https://repositorio.ulima.edu.pe/discover?rpp=10&etal=0&scope=20.500.12724/1409&group_by=none&page={i}"
  resp = requests.get(url)
  soup = BeautifulSoup(resp.text, "html.parser")
  tesis = soup.find_all(class_="row ds-artifact-item ")
  for div in tesis:
    lista_tesis.append("https://repositorio.ulima.edu.pe/"+f"{div.find('a').get('href')}")

In [19]:
lista_tesis

['https://repositorio.ulima.edu.pe//handle/20.500.12724/16321',
 'https://repositorio.ulima.edu.pe//handle/20.500.12724/16334',
 'https://repositorio.ulima.edu.pe//handle/20.500.12724/15986',
 'https://repositorio.ulima.edu.pe//handle/20.500.12724/16306',
 'https://repositorio.ulima.edu.pe//handle/20.500.12724/16267',
 'https://repositorio.ulima.edu.pe//handle/20.500.12724/15791',
 'https://repositorio.ulima.edu.pe//handle/20.500.12724/15631',
 'https://repositorio.ulima.edu.pe//handle/20.500.12724/15579',
 'https://repositorio.ulima.edu.pe//handle/20.500.12724/15610',
 'https://repositorio.ulima.edu.pe//handle/20.500.12724/15636']

In [24]:
import csv
import re

In [25]:
with open("Tesis.csv", "w") as csv_file:
    escritor = csv.writer(csv_file, delimiter=",")
    escritor.writerow(["Institución","Título","Autor", "Grado", "Asesor(es)", "Año", "Resumen", "Categoría"])

for i in lista_tesis:
  resp_t = requests.get(i)
  soup_t = BeautifulSoup(resp_t.text, "html.parser")

  #Título
  try:
    titulo = soup_t.find("h2").text
  except:
    titulo = ""

  #Autor
  try:
    autor_m = []
    a = soup_t.find_all(class_ = "simple-item-view-authors")[0].find_all("div")
    for autor in a:
      autor_m.append(autor.text.strip("\n"))

    
    autor_f = ", ".join(autor_m)
  except:
    autor_f = ""

  #Grado
  try:
    grado_m = soup_t.find_all(class_="simple-item-view-uri")[1].text
    grado = re.findall('\[.*\]', grado_m)[0].strip("[").strip("]")
  except:
    grado = ""

  #Nombre del asesor(es)
  try:
    asesor = []
    b = soup_t.find_all(class_ = "simple-item-view-authors")[1].find_all("div")
    for autor in b:
      asesor.append(autor.text.strip("\n"))
    asesor_f = ", ".join(asesor)
  except:
    asesor_f = ""

  #Año
  try:
    año = str(soup_t.find(class_="simple-item-view-date").find_all(text=True, recursive=False)[1])
  except:
    año = ""

  #Resumen
  try:
    resumen = str(soup_t.find(class_ = "simple-item-view-description").find("div").find_all(text=True, recursive=False)[0])
  except:
    resumen = ""

  #Categoría
  try:
    cat = soup_t.find_all(class_="simple-item-view-uri")[3].find("span").text
  except:
    cat = ""

  x = ["Universidad de Lima", titulo, autor_f, grado, asesor_f, año, resumen, cat]
  with open("Tesis.csv", "a") as csv_file:
    escritor = csv.writer(csv_file, delimiter=",")
    escritor.writerow(x)

## VISUALIZANDO

In [26]:
import pandas as pd

In [27]:
pd = pd.read_csv("Tesis.csv")
pd 

,Institución,Título,Autor,Grado,Asesor(es),Año,Resumen,Categoría
0,Universidad de Lima,Reestructuración del desembarcadero pesquero a...,"Cruz Falcon, Paul, Maldonado Fuentes, Maria An...",Trabajo de suficiencia profesional para optar ...,"Takano Valdivia, Karen",2022,"Actualmente, la infraestructura pesquera artes...",Arquitectura / Urbanismo
1,Universidad de Lima,Recursos personales y resiliencia en adolescen...,"Diaz Sono, Valeria, Zagal Mendoza, Fiorella",Tesis para optar el Título Profesional de Lice...,"Lindo Bárcena, Paola",2022,El presente estudio tiene como objetivos gener...,Psicología / Psicología social
2,Universidad de Lima,Hogar integral multidisciplinario para el adul...,"De La Flor Llona, Sara",Trabajo de suficiencia profesional para optar ...,"More Gómez, Valerie",2022,Esta investigación se basa en entender la situ...,Arquitectura / Proyectos arquitectónicos
3,Universidad de Lima,Estudio de prefactibilidad para la instalación...,"Contreras Flores, Fernando Samir",Tesis para optar el Título Profesional de Inge...,"Bonilla Pastor de Céspedes, Elsie",2022,The main objective of the project is to determ...,Ingeniería industrial / Producción
4,Universidad de Lima,Implementación de un sistema ERP en la empresa...,"Sinforoso Loayza, Manuel Francisco",Trabajo de suficiencia profesional para optar ...,"Ahoki Pajuelo, Carlos Augusto",2022,En el presente trabajo se detalla la mejora qu...,Ingeniería industrial / Tecnología de procesos
5,Universidad de Lima,Estudio de prefactibilidad para la instalación...,"Campos Armijo, Federico Alvaro, Combe Gutierre...",Tesis para optar el Título Profesional de Inge...,"Reaño Vera, Abel Antonio Martin",2022,El presente trabajo consiste en un estudio de ...,Ingeniería industrial / Producción
6,Universidad de Lima,Museo de la Cultura de Tarma: arquitectura par...,"Puente Breña, Danae Ximena",Trabajo de suficiencia profesional para optar ...,"Montestruque Bisso, Octavio Adolfo",2022,El Museo de la Cultura de Tarma es un proyecto...,Arquitectura / Proyectos arquitectónicos
7,Universidad de Lima,Diseño de dispositivo de control de postura sa...,"Cusicanqui Espinoza, Sergio André",Tesis para optar el Título Profesional de Inge...,"Santos Figueroa, Luis Enrique",2022,El presente trabajo de investigación tiene com...,Ingeniería industrial / Diseño e innovación
8,Universidad de Lima,Estudio de prefactibilidad para la instalación...,"Barron Magot, Ana Elvira, Vasquez Quiros, Tito...",Tesis para optar el Título Profesional de Inge...,"Almandoz Nuñez, Ana Maria",2022,Este proyecto consiste en la comercialización ...,Ingeniería industrial / Logística
9,Universidad de Lima,Factores del estancamiento y mortalidad en las...,"Hidalgo Vergara, Sebastián Martin, Xiao, Jianfeng",Tesis para optar el título profesional de Lice...,"Barnett Valdivia, Neptali Jesús",2022,La siguiente investigación tiene objetivo dete...,Ciencias empresariales y económicas / Administ...


---

In [ ]:
pd[pd.Título.str.contains("sistema|sistemas|estadística|modelo|regresión", case=False)]

,Institución,Título,Autor,Grado,Asesor(es),Año,Resumen
4,Universidad de Lima,Implementación de un sistema ERP en la empresa...,"Sinforoso Loayza, Manuel Francisco",Trabajo de suficiencia profesional para optar ...,"Ahoki Pajuelo, Carlos Augusto",2022,En el presente trabajo se detalla la mejora qu...
14,Universidad de Lima,La vida después de la muerte de los miembros d...,"Sevilla Ferrari, Sebastian, Sosa Mendoza, Rosi...",Tesis para optar el Título Profesional de Lice...,"García Contto, José David",2022,El presente trabajo tiene como objetivo compar...
31,Universidad de Lima,Desarrollo profesional hacia el dominio de los...,"Chavez Rivas, Luis Fernando",Trabajo de suficiencia profesional para optar ...,"Lewis Fuentes, Winston",2022,El presente Informe por Suficiencia Profesiona...
79,Universidad de Lima,Relación entre regulación emocional y rendimie...,"Sarmiento Monteverde, Andrea Luciana",Trabajo de suficiencia profesional para optar ...,"Delgado Ramos, Guillermo Enrique",2022,El presente estudio tiene como objetivo analiz...
99,Universidad de Lima,Implementación de un sistema de gas natural po...,"Delgado Leon, Emilio Mario",Trabajo de suficiencia profesional para optar ...,"Sotomayor Cabrera, Arístides",2022,"El presente trabajo tiene como objetivo, la pr..."
189,Universidad de Lima,Mejora de la experiencia de usuario de sistema...,"Chavez Callupe, Paul Jorge",Trabajo de suficiencia profesional para optar ...,"Torres Cáceres, Nolberto",2022,El presente trabajo de investigación es un tra...
203,Universidad de Lima,Proyecto de sistema de control de flujo de trá...,"Rivera Herbozo, Alfredo Henrry",Trabajo de suficiencia profesional para optar ...,"Álvarez Valdivia, Edwin Manuel",2022,La constante demora en la confirmación de la h...
235,Universidad de Lima,Implementación de un sistema de reabastecimien...,"Jimenez Cuadros, Jean Pierre Andre",Trabajo de suficiencia profesional para optar ...,"Vidal Paredes, Alex Antonio",2022,El área de logística de Walsh Perú se encarga ...
237,Universidad de Lima,Sistema de gestión del consumo en tiempo real ...,"Llamas Fernandez, Juan Pablo",Trabajo de suficiencia profesional para optar ...,"Caballero Ortiz, José Alberto",2022,"Desde el inventó de la cocina en el siglo XIX,..."


## Extras:

In [ ]:
#Probando el código para obtener el autor en otras tesis:
resp_t = requests.get("https://repositorio.ulima.edu.pe//handle/20.500.12724/16321")
soup_t = BeautifulSoup(resp_t.text, "html.parser")
autor_m = []
a = soup_t.find_all(class_ = "simple-item-view-authors")[0].find_all("div")
for autor in a:
  autor_m.append(autor.text.strip("\n"))

autor_m

['Cruz Falcon, Paul', 'Maldonado Fuentes, Maria Angela']